### Experiment with MongoEngine

The ODM we use to access MongoDB

In [1]:
from dexter.DB import DB, Account, Entry, Transaction

Open the database:

In [2]:
DB.open('foo')

Make an account:

In [3]:
acct = Account(name='equity', group='equity')

Save it:

In [4]:
acct.save()

<Account: Account object>

If we open that DB with `mongosh` we should see the account.

```
$ mongosh

test> use foo
switched to db foo

foo> db.account.find()
[
  {
    _id: ObjectId('67c61fa19d0161a19b80469e'),
    name: 'equity',
    group: 'equity'
  }
]
```

It worked!  🎉

### Contents of a Collection

In [5]:
Account.objects

[<Account: Account object>]

In [6]:
Account.objects[0]

<Account: Account object>

In [7]:
acct = Account.objects[0]

In [8]:
acct.name

'equity'

### Low Level API

We can also connect to the DB directly to use the `pymongo` library, _e.g._ to get collection names.

After calling `DB.open` we can get a reference to the client and the current database using static vars of the module:

In [9]:
DB.client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None))

In [10]:
DB.database

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'foo')

In [11]:
db = DB.database

In [12]:
db.account

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'foo'), 'account')

In [13]:
db.account.find_one()

{'_id': ObjectId('67ca0417b64ec7443ae9a2d8'),
 'name': 'equity',
 'group': 'equity'}

### The Big Picture

Use the high level API when working with data.  MongoEngine converts the documents into objects (which is something we'd be doing ourselves if we didn't use it).

Use the low level API for collective operations: exporting, importing, ...

**NOTE**  It's possible to get a document using the low level API, as shown above, but it will be a `dict`, not a model instance.

### Transactions

In [14]:
t = Transaction(description='hi', comment='aloha')

In [15]:
t.description

'hi'

Nice -- the list fields are initially empty.

In [16]:
t.tags

[]

In [17]:
t.entries

[]

### Entries

In [18]:
p = Entry(uid='xxx', etype='credit', date='2025-03-05', amount=1000, account='unknown')

In [19]:
p

<Entry: Entry object>

In [20]:
p.etype

<EntryType.cr: 'credit'>

In [21]:
p.amount

1000.0

### References

The big test -- can we add that Entry to the transaction?

In [22]:
t.entries.append(p)

In [23]:
t.entries

[<Entry: Entry object>]

Yes!  🎉

### Misc Commands

In [24]:
db.stats

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'foo'), 'stats')

In [26]:
db.stats.find_one

<bound method Collection.find_one of Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'foo'), 'stats')>

In [27]:
db.list_collection_names()

['account']

In [28]:
db.command('count','account')

{'n': 1, 'ok': 1.0}

In [34]:
db.command('hello')

{'isWritablePrimary': True,
 'topologyVersion': {'processId': ObjectId('67c4fbece8eb6b0fbfe3b917'),
  'counter': 0},
 'maxBsonObjectSize': 16777216,
 'maxMessageSizeBytes': 48000000,
 'maxWriteBatchSize': 100000,
 'localTime': datetime.datetime(2025, 3, 6, 20, 27, 1, 115000),
 'logicalSessionTimeoutMinutes': 30,
 'connectionId': 294,
 'minWireVersion': 0,
 'maxWireVersion': 25,
 'readOnly': False,
 'ok': 1.0}

In [35]:
db.command('hostInfo')

{'system': {'currentTime': datetime.datetime(2025, 3, 6, 20, 28, 42, 332000),
  'hostname': 'cthulhu.local',
  'cpuAddrSize': 64,
  'memSizeMB': 65536,
  'memLimitMB': 65536,
  'numCores': 12,
  'numCoresAvailableToProcess': 12,
  'numPhysicalCores': 12,
  'numCpuSockets': 1,
  'cpuArch': 'arm64',
  'numaEnabled': False,
  'numNumaNodes': 1},
 'os': {'type': 'Darwin', 'name': 'Mac OS X', 'version': '24.3.0'},
 'extra': {'versionString': 'Darwin Kernel Version 24.3.0: Thu Jan  2 20:24:23 PST 2025; root:xnu-11215.81.4~3/RELEASE_ARM64_T6020',
  'alwaysFullSync': 0,
  'nfsAsync': 0,
  'model': 'Mac14,13',
  'cpuString': 'Apple M2 Max',
  'pageSize': 16384,
  'scheduler': 'edge'},
 'ok': 1.0}

In [36]:
db.command('ping')

{'ok': 1.0}